### 1 切换到项目路径

In [1]:
%cd ~/vits-zh  # 具体目录看实际情况调整
%ls

/home/featurize/vits-zh


In [2]:
import torch
import commons
import utils
from models import SynthesizerTrn
from text import tokens2ids, pypinyin_g2p_phone, symbols
from scipy.io.wavfile import write
from IPython.display import display, Audio

In [4]:
hps = utils.get_hparams_from_file("./configs/singlespeaker.json")  # 这里需要换成自己的配置文件

net_g = SynthesizerTrn(  # 初始化模型
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)

# 加载训练好的模型，一般有400M，但生成模型只占其中的150M。其它是梯度优化器的占用空间，只在训练时有用
_, _, _, epoch_str = utils.load_checkpoint('/home/featurize/work/logs/paimon/G_230000.pth', net_g, None)

INFO:root:Loaded checkpoint '/home/featurize/work/logs/paimon/G_230000.pth' (iteration 589)


In [51]:
# torch.save(net_g.state_dict(), './vocoder.pth')     # 用这个将150M的模型单独提取出来
# net_g.load_state_dict(torch.load('./vocoder.pth'))  # 读取测试

<All keys matched successfully>

In [5]:
def gen_wave(in_text, sid):
    phonemes = pypinyin_g2p_phone(in_text)
    input_ids = torch.LongTensor(tokens2ids(phonemes))
    with torch.no_grad():
        x_tst = input_ids.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([input_ids.size(0)])
        sid = torch.LongTensor([sid])
        audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=0, noise_scale_w=0, length_scale=1)[0][0,0].data.cpu().float().numpy()
    display(Audio(audio, rate=hps.data.sampling_rate))

In [6]:
gen_wave("派蒙，不要再打电动了哦。", 1)

In [16]:
gen_wave("我的声音，听起来怎么样？", 0)